# Additional installation instructions for molecules

In order to use the graph models the FlowMol library needs to be installed. Check FlowMol **v2** here: https://github.com/Dunni3/FlowMol/tree/v2.0 for instructions on how to setup the FlowMol environment, or perform the following steps:

- Checkout the FlowMol repo (making sure to use the v2 version, use the link above)
- Run the build_flowmol_conda.sh script in this folder (the `notebooks` folder) from the base directory of the FlowMol repo **with the genexp environment activated** after performing the installation instructions in this repo's README in order to build FlowMol into the genexp environment.
- Install the QM9/GEOM datasets into the FlowMol repo using the instructions in the FlowMol repo depending on which model you want to use. The 

The rest of this notebook should run successfully after performing those steps.

In [1]:
from tqdm.notebook import tqdm
from genexp.trainers.genexp import FDCTrainerFlow
from genexp.utils import seed_everything

In [2]:
from genexp.graph_flow import GraphFlowModel, GraphEulerMaruyamaSampler
import flowmol
import torch

RuntimeError: operator torchvision::nms does not exist

## Loading base model

In [ ]:
flowmol_model = flowmol.load_pretrained('qm9_ctmc') # Change to geom_ctmc for GEOM
model = GraphFlowModel(flowmol_model)
sampler = GraphEulerMaruyamaSampler(model, sampler_type='euler')

## Fine-tuning with FDC 

In [ ]:
from omegaconf import OmegaConf
import copy

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
config = OmegaConf.load('../configs/example_cvar.yaml')
model = model.to(device)

seed_everything(config.seed)
fdc_trainer = FDCTrainerFlow(config, copy.deepcopy(model), copy.deepcopy(model), device=device, sampler=sampler)

for k in tqdm(range(config.num_md_iterations)):
    for i in range(config.adjoint_matching.num_iterations):
        am_dataset = fdc_trainer.generate_dataset()
        fdc_trainer.finetune(am_dataset, steps=config.adjoint_matching.finetune_steps)

    fdc_trainer.update_base_model()